In [1]:
import numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import zarr
from numcodecs import Blosc
import torch
import torch.nn as nn
import ResNet as ResNet 
import torchvision.transforms as transforms
import torch.nn.functional as F
import re
from imgaug import augmenters as iaa
import argparse
import json

In [2]:
def make_parse():
    parser = argparse.ArgumentParser()
    parser.add_argument('--augment', default=False, action='store_true')
    
    args = parser.parse_args()
    return args

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def get_coords(batch_names): #ToDO: Change function for precise coords
    coords = []
    
    for tile_name in batch_names: 
        # print(tile_name)
        pos = re.findall(r'\((.*?)\)', tile_name)
        x, y = pos[-1].split('_')
        coords.append((int(x),int(y)))
    return coords

def iaa_augment(img):

    sometimes = lambda aug: iaa.Sometimes(0.5, aug, name="Random1")
    sometimes2 = lambda aug: iaa.Sometimes(0.2, aug, name="Random2")
    sometimes3 = lambda aug: iaa.Sometimes(0.9, aug, name="Random3")
    sometimes4 = lambda aug: iaa.Sometimes(0.9, aug, name="Random4")
    sometimes5 = lambda aug: iaa.Sometimes(0.9, aug, name="Random5")

    transforms = iaa.Sequential([
        iaa.AddToHueAndSaturation(value=(-30, 30), name="MyHSV"), #13
        sometimes2(iaa.GammaContrast(gamma=(0.85, 1.15), name="MyGamma")),
        iaa.Fliplr(0.5, name="MyFlipLR"),
        iaa.Flipud(0.5, name="MyFlipUD"),
        sometimes(iaa.Rot90(k=1, keep_size=True, name="MyRot90")),
        iaa.OneOf([
            sometimes3(iaa.PiecewiseAffine(scale=(0.015, 0.02), cval=0, name="MyPiece")),
            sometimes4(iaa.ElasticTransformation(alpha=(100, 200), sigma=20, cval=0, name="MyElastic")),
            sometimes5(iaa.Affine(scale={"x": (0.95, 1.05), "y": (0.95, 1.05)}, rotate=(-45, 45), shear=(-4, 4), cval=0, name="MyAffine"))
        ], name="MyOneOf")
    ])
    seq_img_d = transforms.to_deterministic()
    img = seq_img_d.augment_image(img)

    return img


In [4]:
torch.set_num_threads(8)
torch.manual_seed(2022)

args = make_parse()

augment=args.augment
print('Augment Data: ', augment)

home = Path.cwd().parts[1]
data_root = Path(f'/{home}/ylan/data/DeepGraft/tissue_detection/224_128uM/images')
slide_patient_path = f'/{home}/ylan/DeepGraft/training_tables/slide_patient_dict.json'
cohort_stain_path = f'/{home}/ylan/DeepGraft/training_tables/cohort_stain_dict.json'
with open(slide_patient_path, 'r') as f:
    slide_patient_dict = json.load(f)
with open(cohort_stain_path, 'r') as f:
    cohort_stain_dict = json.load(f)
# output_path = Path(f'/{home}/ylan/wsi_tools/debug/zarr')
# cohorts = ['DEEPGRAFT_RU'] #, 
# cohorts = ['Aachen_Biopsy_Slides'] #, 
# cohorts = ['Aachen_Biopsy_Slides', 'DEEPGRAFT_RU', 'DEEPGRAFT_RA', 'Leuven'] #, 
compressor = Blosc(cname='blosclz', clevel=3)

val_transforms = transforms.Compose([
        # 
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
        # RangeNormalization(),
    ])


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
scaler = torch.cuda.amp.GradScaler()
n_classes = 2
# out_features = 1024
model_ft = ResNet.resnet50(num_classes=128, mlp=False, two_branch=False, normlinear=True)

model_ft.fc = nn.Identity()
# print(model_ft)
# model_ft.fc = nn.Linear(2048, out_features)
home = Path.cwd().parts[1]
model_ft.load_state_dict(torch.load(f'/{home}/ylan/workspace/TransMIL-DeepGraft/code/models/ckpt/retccl_best_ckpt.pth'), strict=True)
# for param in model_ft.parameters():
#     param.requires_grad = False
# for m in model_ft.modules():
#     if isinstance(m, torch.nn.modules.batchnorm.BatchNorm2d):
#         m.eval()
#         m.weight.requires_grad = False
#         m.bias.requires_grad = False
# model_ft.fc = nn.Linear(2048, out_features)
model_ft.eval()
model_ft.to(device)

batch_size = 100

usage: ipykernel_launcher.py [-h] [--augment]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9040 --control=9038 --hb=9037 --Session.signature_scheme="hmac-sha256" --Session.key=b"fea321c1-c51d-4123-ac4e-21d7b6f0be68" --shell=9039 --transport="tcp" --iopub=9041 --f=/home/ylan/.local/share/jupyter/runtime/kernel-v2-8466dCu9m1xIy2SG.json


SystemExit: 2

In [ ]:
patient_cohort_dict = {}
for cohort in cohort_stain_dict.keys():
    cohort_patient_list = list(cohort_stain_dict[cohort].keys())
    for patient in cohort_patient_list:
        patient_cohort_dict[patient] = cohort

: 

In [ ]:
for f in data_root.iterdir():
    slide_name = f.stem.split('_', 1)[0]
    patient = slide_patient_dict[slide_name]